# Cleaning PI Info
In order to prevent duplicate information used to train a model, store PI information separately from list of features.

**Eventually store as SQL database**

In [1]:
import pandas as pd
import numpy as np
from random import randint
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import cleaning_strings as cln
import importlib as imp
imp.reload(cln);

## Local functions 

In [2]:
def add_zero_zip(df, length, col1 = 'org_zipcode', col2 = 'org_country', country = 'united states'):
    
    '''Add leading zero to New England zipcodes.
    Arguments are the df and length of incorrect zipcode.
    Returns two lists, one with the original zipcodes and one with the added leading zero.'''
    
    original = []
    actual = []
    for num in df.ix[(df[col1].str.len() == length) & (df[col2].str.contains(country))][col1]:
        original.append(num)
        num = '0' + num
        actual.append(num)
    return original, actual

def add_zero_duns(df, col1 = 'org_duns'):
    
    '''Add leading zeros to DUNS numbers until DUNS number is 9 digits.
    Returns the df with leading zeros added to short DUNS numbers.'''
    
    actual = []
    series = df[col1].tolist()
    for num in series:
        
        #skip null entries
        if type(num) == float:
            pass
        
        #add zeros until length is 9
        else:
            while len(num) < 9.0:
                num = '0' + num
        actual.append(num)
    
    #add new column to df with 9 digit DUNS numbers
    df[col1 + '_'] = actual
    
    #delete original column and rename new column
    del df[col1]
    df = df.rename(columns = {col1 + '_':col1})
    return df

def get_unique_values(df, name, col_filter = 'org_duns'):
    
    '''Get unique values in each column to be filled for a particular entry.
    Takes a df, the name of the column to be filtered, and the name of the entry.
    Returns the dictionary where the keys are the columns to be filled and the
    values are the unique values for the entry.
    Also returns a list of the non-unique organization names that match an individual DUNS number.'''
    
    #create list of columns to fill
    cols_to_fill = 'org_duns org_name org_city org_state org_zipcode org_country'.split()
    
    #create dictionary where keys are the column names
    cols_dict = {key: None for key in cols_to_fill}
    
    #filter rows by element (name) in column to be filtered (col_filter)
    values = df.ix[df[col_filter] == name]
    
    #want to match each column name (key) with the correct entry (value)
    for col in cols_to_fill:
        
        #get the unique values of each column to fill
        unique_value = values[col].dropna().unique()
        
        #check there is only 1 unique value and match with key
        if unique_value.shape == (1,):
            cols_dict[col] = unique_value[0]
        
        #check if array is empty
        elif unique_value.shape == (0,):
            cols_dict[col] = np.nan
        
        else:
            #fill value with most frequently occurring value
            #assumption is most frequently occurring value is the correct value
            cols_dict[col] = values[col].value_counts().idxmax()
            
            #print out org_names to check whether individual DUNS match multiple org_names
            #for diagnostic purposes
            #if col == 'org_name':
                #print(unique_value)
    return cols_dict

def fill_missing(df, col_filter = 'org_duns'):
    
    '''Fill missing values in each grant based on the unique DUNS number.
    Returns a df.'''
    
    #get list of unique entries (default is unique DUNS numbers)
    list_unique = df[col_filter].dropna().unique().tolist()
    df_new = pd.DataFrame(columns = df.columns)
    
    #for every DUNS number:
    for i in list_unique:
        #fill null values in each column with corresponding unique values
        to_fill = get_unique_values(df, i, col_filter)
        df_append = df.ix[df[col_filter] == i].fillna(to_fill)
        df_new = df_new.append(df_append)
    df_new = df_new.sort_index()
    return df_new

def merge_fill_nans(df1, df2, col):
    
    '''Merge two dfs and fill NaNs in the first df with values from the duplicated column
    in the second df. Drop the duplicate column.
    Returns a df.'''
    
    #merge the original df and the new df and fill NaNs
    df_merged = pd.merge(df1, df2, how = 'left', left_index = True, right_index = True, suffixes = ('', '_copy'))
    df_merged[col].fillna(df_merged[col + '_copy'], inplace = True)
    
    #remove duplicate columns
    to_keep = [col for col in df1.columns if '_copy' not in col]
    df_merged = df_merged[to_keep]
    return df_merged

def add_dummy_duns(df, col_filter1 = 'org_duns', col_filter2 = 'org_name'):
    
    '''Fill missing DUNS numbers with dummy 9 digit number.
    Returns a df with all DUNS filled in.'''
    
    #extract entries where DUNS is not listed
    nulls = df.ix[df[col_filter1].isnull()]
    null_names = nulls[col_filter2].unique().tolist()
    df_replace = []
    
    #for each institute where the DUNS is not listed
    for i in null_names:
        #generate a random DUNS number; no original DUNS is listed with 4 leading zeros.
        dummy = '0000' + str(randint(10000, 99999))
        #create a series where the DUNS number per institute is filled in
        df_fill = pd.DataFrame(df.ix[df[col_filter2] == i][col_filter1].fillna(dummy))
        df_replace.append(df_fill)
    
    #concatenate the list of series into a dataframe
    df_replace = pd.concat(df_replace)
    df_merged = merge_fill_nans(df, df_replace, col_filter1)
    return df_merged

def replace_info(df, col_filter = 'org_duns', all_cols = ['org_name', 'org_city',
                                                          'org_state', 'org_zipcode']):
    
    '''Replace incorrect information.'''
    
    duns_list = df[col_filter].unique().tolist()
    df_new = []
    for duns in duns_list:
        subset = df.ix[df[col_filter] == duns]
        for col in all_cols:
            values = subset[col].value_counts(dropna = False)
            to_replace = values.index.tolist()
            replacement = values.idxmax()
            subset[col] = subset[col].replace(to_replace, replacement)
        df_new.append(subset)
    df_new = pd.concat(df_new)
    df_new.sort_index(inplace = True)
    return df_new

def replace_nih_info(df, dict_, org_name = 'org_name'):
    
    '''Some NIH centers have updated names.
    Replace the appropriate grants with this information.
    Returns a df.'''
    
    for k, v in dict_.items():
        df.replace({k:v}, inplace = True)
        df.ix[df[org_name] == v] = df.ix[df[org_name] == v].fillna(method = 'bfill').fillna(method = 'ffill')
    return df

def add_nih_info(df, col, replace_with, org_name = 'org_name', org_country = 'org_country'):
    
    '''Add missing NIH information. These are NIH centers where no information is listed other
    than the organization name.
    Returns a df.'''
    
    #NIH institutes are those where the org_name is listed but the country is not
    df_replace = pd.DataFrame(df.ix[~df[org_name].isnull() & df[org_country].isnull()]\
                              [col].fillna(replace_with))
    df_merged = merge_fill_nans(df, df_replace, col)
    return df_merged

## Cleaning PI information
We want a dataframe where each row is a single PI (no duplicates) and associated organization information as a cross-reference to the grants data.

Import relevant columns from csv with raw grant information.

In [3]:
columns = 'fy pi_ids pi_names org_name org_city org_state org_country org_zipcode org_duns'.split()
dtypes = {key: str for key in columns}

In [4]:
pi_info_raw = pd.read_csv('all_grants.csv', compression = 'gzip', usecols = columns, dtype = dtypes)

#Only analyzing grants from 2000 onwards, as prior to that no funding information is available
pi_info_raw = pi_info_raw.ix[pi_info_raw['fy'] >= '2009']
pi_info_raw.tail()

,fy,org_city,org_country,org_duns,org_name,org_state,org_zipcode,pi_ids,pi_names
2223287,2016,lawrence,united states,76248616,university of kansas lawrence,ks,660457568,8097,"mcgill, jodi l.;"
2223288,2016,davis,united states,47120084,university of california at davis,ca,956186153,8097,"clancy, colleen e;"
2223289,2016,la jolla,united states,804355790,university of california san diego,ca,920930934,8097,"feng, gen-sheng ;"
2223290,2016,coral gables,united states,52780918,university of miami school of medicine,fl,331462926,8097,"sharifai, nima ;"
2223291,2016,toledo,united states,51623734,university of toledo,oh,436063390,8097,"liu, song-tao ;"


In [5]:
pi_info_cleaned = cln.strip_df(pi_info_raw, ' ', ';', ' ', '.')
pi_info_cleaned.tail()

,fy,org_city,org_country,org_duns,org_name,org_state,org_zipcode,pi_ids,pi_names
2223287,2016,lawrence,united states,76248616,university of kansas lawrence,ks,660457568,8097,"mcgill, jodi l"
2223288,2016,davis,united states,47120084,university of california at davis,ca,956186153,8097,"clancy, colleen e"
2223289,2016,la jolla,united states,804355790,university of california san diego,ca,920930934,8097,"feng, gen-sheng"
2223290,2016,coral gables,united states,52780918,university of miami school of medicine,fl,331462926,8097,"sharifai, nima"
2223291,2016,toledo,united states,51623734,university of toledo,oh,436063390,8097,"liu, song-tao"


In [6]:
pi_info_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 653801 entries, 1569491 to 2223291
Data columns (total 9 columns):
fy             653801 non-null object
org_city       625577 non-null object
org_country    625632 non-null object
org_duns       611923 non-null object
org_name       653128 non-null object
org_state      617661 non-null object
org_zipcode    622758 non-null object
pi_ids         653801 non-null object
pi_names       653801 non-null object
dtypes: object(9)
memory usage: 49.9+ MB


## Splitting PI Info
Some grants have multiple PIs listed on the grant, and the information for the group of PIs is listed only as the contact PI's information. In order to get unique PI information, these PI groups must be separated into individual PIs.

Split pi_info_cleaned into two dataframes, one containing grouped (multiple) PIs and one containing single PIs.

In [11]:
multi_pi_unsplit = pi_info_cleaned.ix[pi_info_cleaned['pi_ids'].str.contains('contact', na = False)]
pi_info = pi_info_cleaned.drop(multi_pi_unsplit.index)
pi_info.shape

(653801, 9)

Drop duplicates to obtain unique PIs that were awarded solo grants. Check both PI IDs and organization name to check for PIs that moved to a different institution.

In [8]:
pi_unique = pi_info.drop_duplicates(['pi_ids', 'org_duns'])
pi_unique.shape
pi_unique.tail()

(9615, 9)

,fy,org_city,org_country,org_duns,org_name,org_state,org_zipcode,pi_ids,pi_names
2222317,2016,NaN,NaN,437306274,NaN,NaN,NaN,8097,"cucca, francesco"
2222906,2016,mansfield,united states,940639748,"agypharma, llc",tx,760633809,8097,"nguyen, vien"
2222972,2016,pickerington,united states,74859047,ohio state bureau/workers' compensation,oh,431478310,8097,"al-tarawneh, ibraheem"
2223034,2016,novato,united states,79336779,image analyst software,ca,949451775,8097,"gerencser, akos a"
2223094,2016,san diego,united states,78884924,"forge therapeutics, inc",ca,921211126,8097,"puerta, david"


Split multiple PIs on PI ID.

In [12]:
multi_pi_unsplit.head()

,fy,org_city,org_country,org_duns,org_name,org_state,org_zipcode,pi_ids,pi_names


In [9]:
multi_pi = cln.split_rows(multi_pi_unsplit, 'pi_ids', by = ';')
multi_pi.tail()

AttributeError: 'Series' object has no attribute 'stack'

Create a df where the '(contact)' string has been stripped from pi_ids; this will allow identification of unique PI IDs (otherwise an ID with '(contact)' appended at the end is viewed as a unique string).

In [ ]:
multi_stripped = multi_pi.copy()
multi_stripped['pi_ids'] = multi_stripped['pi_ids'].str.strip(' (contact)')
multi_stripped.tail()

Create columns in split multi_pi df denoting whether the PI is the contact and whether the PI ID is already present in the df pi_unique (which contains all PIs that are solo authors of a grant). If the PI is already in pi_unique, we do not need to separate their information again.

In [ ]:
multi_pi['contact'] = multi_pi['pi_ids'].str.contains('contact', na = False)
in_pi_unique = multi_stripped['pi_ids'].isin(pi_unique['pi_ids'])
multi_pi['unique_pi'] = in_pi_unique
multi_pi.tail()

One PI is always designated as a contact if there are multiple PIs listed on a grant. If the PI's ID was not in the df pi_unique, but is listed as a contact (that is, unique_pi == False but contact == True), then we can isolate their information and add this information to pi_unique.

In [ ]:
unique_contact = multi_pi[(multi_pi['unique_pi'] == False) & (multi_pi['contact'] == True)]
unique_contact = cln.split_rows(unique_contact, 'pi_names', ';')
unique_contact = unique_contact[unique_contact['pi_names'].str.contains('contact')]
unique_contact = cln.strip_series(unique_contact, ['pi_ids', 'pi_names'])

#shape before dropping duplicates
unique_contact.shape
unique_contact = unique_contact.drop_duplicates('pi_ids org_duns'.split())

#shape after dropping duplicates
unique_contact.shape
unique_contact.tail()

If the PI ID is neither listed in unique_pi nor are they ever listed as a contact, then we cannot identify whether their organization information is actually different from the contact PI's information. These names will therefore not be split (the IDs are already split).

**Note:** When doing analysis from multiple years, PI IDs should be cross-referenced across years in case a PI did have a solo grant in one year but not in others.

In [ ]:
not_contact = multi_pi[(multi_pi['unique_pi'] == False) & (multi_pi['contact'] == False)]
not_contact = not_contact.drop_duplicates('pi_ids org_duns'.split())
not_contact.shape
not_contact.tail()

Join all dfs containing unique PI ids.

In [ ]:
#Reorder columns so joins can be performed correctly

#unique PIs with a solo grant
cols1 = unique_contact.columns.tolist()
cols1 = cols1[-4:-3] + cols1[-1:] + cols1[0:5]
unique_1 = unique_contact[cols1]

#unique PIs that neither have a solor grant nor are listed as a contact
cols2 = not_contact.columns.tolist()
cols2 = cols2[-3:-2] + cols2[0:6]
unique_2 = not_contact[cols2]

#unique PIs that do not have solo grants but are listed as a contact
unique_multi = unique_1.append(unique_2)
unique_multi = unique_multi.drop_duplicates('pi_ids org_name'.split())
unique_multi.shape

pi_unique.shape
pi_unique = pi_unique.append(unique_multi)

pi_unique = cln.strip_series(pi_unique, ['pi_ids'], strip = ' ')
pi_unique = pi_unique.drop_duplicates('pi_ids org_name'.split())
pi_unique.shape

## Fixing zipcodes
The leading zero of New England zipcodes was dropped in the raw data. Add leading zero to zipcodes from the US that are length 8 or 4.

In [ ]:
pi_unique['org_zipcode'].str.len().value_counts()

In [ ]:
zip_8, zip_9 = add_zero_zip(pi_unique, 8.0)
zip_4, zip_5 = add_zero_zip(pi_unique, 4.0)

In [ ]:
pi_unique['org_zipcode'] = pi_unique['org_zipcode'].replace(zip_8, zip_9)
pi_unique['org_zipcode'] = pi_unique['org_zipcode'].replace(zip_4, zip_5)

In [ ]:
pi_unique['org_zipcode'].str.len().value_counts()

## DUNS numbers
The DUNS number should be a 9-digit number that uniquely identifies an organization. However, the DUNS numbers of organizations changed between 2008 to 2009.

In [ ]:
pi_unique.ix[(pi_unique['org_name'].str.contains('stanford', na = False)) & ((pi_unique['fy'] == '2008') | (pi_unique['fy'] == '2009'))]

The DUNS number is also not unique between 2008 and 2009.

In [ ]:
pi_unique.ix[pi_unique['org_duns'] == '009214214']['org_name'].unique()

Organization names and DUNS change between 2008 and 2009, and the DUNS are not unique between the two epochs. It will be difficult to cross reference each institution across years. For the first pass, we will only analyze grants from 2009 and later.

In [ ]:
pi_unique = pi_unique.ix[pi_unique['fy'] > '2008']

### DUNS numbers of different lengths
DUNS numbers should be 9 digits.

In [ ]:
pi_unique['org_duns'].str.len().value_counts()

In [ ]:
pi_unique.ix[pi_unique['org_duns'].str.len() == 8.0].head()

In [ ]:
pi_unique.ix[pi_unique['org_duns'].str.len() == 7.0].head()

It seems like most of the 8 and 7 digit DUNS numbers are missing leading zeros. Assume this is the case and add leading zeros to DUNS numbers.

In [ ]:
pi_unique = add_zero_duns(pi_unique)

Re-organize columns.

In [ ]:
old_cols = pi_unique.columns.tolist()
new_cols = old_cols[0:1] + old_cols[-3:-1] + old_cols[-1:] + old_cols[3:4] + old_cols[1:2] + old_cols[4:5] + old_cols[5:6] + old_cols[2:3]
pi_unique = pi_unique[new_cols]
pi_unique.head()

According to NIH's description, multiple DUNS are separated by a semi-colon. There are only two groups where multiple DUNS are listed, those with 19 characters and those with 20 characters.

In [ ]:
pi_unique.ix[pi_unique['org_duns'].str.len() == 20.0].head()

In [ ]:
pi_unique.ix[pi_unique['org_duns'].str.len() == 19.0].head()

For organizations where two DUNS are listed, it seems that both DUNS are used to identify the organization. The only difference between a DUNS with 20 characters and 19 characters is a space after the ';'. Add a space to the 19-length DUNS and replace these values in the df.

In [ ]:
#Get the DUNS numbers from pi_unique where the length of the DUNS is 19
len19 = pi_unique.ix[pi_unique['org_duns'].str.len() == 19.0]['org_duns'].values.tolist()

#add a space after the semi-colon
len19_new = []
for i in range(len(len19)):
    len19_new.append(len19[i][:10] + ' ' + len19[i][10:])

#replace the original DUNS of 19 characters
pi_unique['org_duns'] = pi_unique['org_duns'].replace(len19, len19_new)

Remove duplicates by pi_id and org_duns, as a PI can move between institutions.

In [ ]:
pi_unique.shape
pi_unique = pi_unique.drop_duplicates('pi_ids org_duns'.split())
pi_unique.shape

# Missing information
First check the entries where no country is listed.

In [ ]:
pi_unique.ix[pi_unique['org_country'].isnull()]['org_name'].value_counts(dropna = False)

* 'clinical sciences' and 'clinical center' likely refer to the same center, as clinical sciences is listed in years < 2012 and afterwards clinical center is listed.
* Replace 'environmental health sciences' with 'national institute of environmental health sciences'
* Correct single entries
* Drop the 'children s hospital medic' entry.

In [ ]:
a = ['clinical sciences', 'environmental health sciences', 'iit resch institute',
    'institute for defense ana', 'california univ los angel']
b = ['clinical center', 'national institute of environmental health sciences', 'iit research institute',
    'institute for defense analyses, inc', 'university of california los angeles']
pi_unique.replace(a, b, inplace = True)

In [ ]:
pi_unique.ix[pi_unique['org_name'].str.contains('danmarks tekniske univers', na = False)] = \
pi_unique.ix[pi_unique['org_name'].str.contains('danmarks tekniske univers', na = False)]\
.fillna({'org_city':'Kongens Lyngby', 'org_country':'Denmark'}, inplace = True)

No information available for the entry 'children s hospital medic'

In [ ]:
to_drop = pi_unique.ix[pi_unique['org_name'] == 'children s hospital medic'].index
pi_unique.drop(to_drop, inplace = True)

## Filling missing organization information
Based on unique DUNS numbers, fill in:
* Organization name
* City, state, zipcode
* Country

To fill NaNs, we are filtering by DUNS numbers. Therefore first pull out organizations where the DUNS is not listed.

In [ ]:
null_duns = pi_unique.ix[pi_unique['org_duns'].isnull()]

In [ ]:
pi_unique.isnull().sum()

In [ ]:
pi_unique = fill_missing(pi_unique)

In [ ]:
pi_unique.isnull().sum()

## Replace information
A lot of entries have incorrect information entered (wrong zip codes in particular). We assume that the mistakes are less common than the correct entry. Therefore find the most common entry and fill the other entries accordingly.

In [ ]:
#pi_unique.to_csv('intermediate.csv', index = False, compression = 'gzip')
#pi_unique = pd.read_csv('intermediate.csv', compression = 'gzip')

In [ ]:
pi_unique = replace_info(pi_unique)

## Adding dummy DUNS numbers
First add back grants where the DUNS number was not listed.

In [ ]:
pi_unique.shape

In [ ]:
pi_unique = pi_unique.append(null_duns)
pi_unique.sort_index(inplace = True)
pi_unique.shape

In [ ]:
null_duns.isnull().sum()
pi_unique.isnull().sum()

Clean organization names by adding a space after the '&' symbol.

In [ ]:
and_names = pd.DataFrame()
and_names['original'] = pi_unique.loc[pi_unique['org_name'].str.contains(' &[^ ]', na = False)]['org_name']
and_names['with_space'] = pi_unique.loc[pi_unique['org_name'].str.contains(' &[^ ]', na = False)]['org_name']\
.str.replace('&', '& ')

and_names = and_names.drop_duplicates()

no_space = and_names['original'].tolist()
with_space = and_names['with_space'].tolist()
pi_unique.replace(no_space, with_space, inplace = True)

Fill in missing DUNS numbers for those entries where the DUNS number is listed in other rows.

In [ ]:
pi_unique = fill_missing(pi_unique, col_filter = 'org_name')

In [ ]:
pi_unique.isnull().sum()

Add dummy DUNS numbers to those organizations where the DUNS are not listed. **Caveat:** If the organization name is misspelled or incorrectly entered, then a unique DUNS will be generated for that entry even though the organization is already listed in the organization. Matching by string similarity will be difficult. Matching by zip codes are also unreliable because zip codes seem particularly prone to incorrect entry (see below).

In [ ]:
pi_unique = add_dummy_duns(pi_unique)

In [ ]:
pi_unique.isnull().sum()

## Missing NIH information

Most grants without a listed country seem to be associated with different NIH centers. Most information of these organizations, that is, DUNS number, city, state and zipcode in addition to country, are missing. There are some NIH centers where all this information is available; however, the name of the center is listed differently.

In [ ]:
pi_unique.ix[pi_unique['org_country'].isnull()]['org_name'].value_counts(dropna = False)

In [ ]:
pi_unique.ix[pi_unique['org_zipcode'].str.contains('20892', na = False)]['org_name'].value_counts()

Replace/fill the corresponding NIH centers in the pi_unique dataframe with the information above.

In [ ]:
dict_nih_names = {
    'national institute of allergy and infectious diseases':'u.s. national inst allergy & infect dis',
    'child health and human development':'u.s. national inst/child hlth/human dev',
    'diabetes, digestive, kidney diseases':'u.s. national inst diabetes/digst/kidney',
    'national institute of mental health':'u.s. national institute of mental health',
    'neurological disorders and stroke':'u.s. national inst/neuro/ds/stroke',
    'alcohol abuse and alcoholism':'u.s. national inst alcohol ab/alcoholism',
    'heart, lung, and blood institute':'u.s. national heart lung and blood inst'
}

pi_unique = replace_nih_info(pi_unique, dict_nih_names)

### Adding missing NIH information

In [ ]:
pi_unique.ix[pi_unique['org_country'].isnull()]['org_name'].value_counts(dropna = False)

No information about NIH city, state, country and zipcode is listed for the remaining NIH centers. Add this information as bethesda, md, united states and 20892, respectively. **Note:** This address may not be exact, as some NIH institutes/centers may be located elsewhere, but this information will represent general NIH information.

In [ ]:
pi_unique = add_nih_info(pi_unique, col = 'org_city', replace_with = 'bethesda')
pi_unique = add_nih_info(pi_unique, col = 'org_state', replace_with = 'md')
pi_unique = add_nih_info(pi_unique, col = 'org_zipcode', replace_with = '20892')
pi_unique = add_nih_info(pi_unique, col = 'org_country', replace_with = 'united states')

In [ ]:
pi_unique.isnull().sum()

### Missing zip codes for USA entries

In [ ]:
null_zip = pi_unique.ix[(pi_unique['org_country'] == 'united states') & pi_unique['org_zipcode'].isnull()]

In [ ]:
null_zip.count()

In [ ]:
null_zip['org_duns'].value_counts().count()

In [ ]:
null_zip

It may be possible to fill the missing zip codes via an automated Google search (see http://stackoverflow.com/questions/37083058/programmatically-searching-google-in-python-using-custom-search). This will likely be unnecessary because only about 400 unique organizations are missing zip codes. When we analyze and/or plot geospatial data we can decide whether to fill in these values.

## Final information 

In [ ]:
pi_unique.head(10)

In [ ]:
pi_unique.tail()

In [ ]:
pi_unique.to_csv('pi_info.csv', index = False, compression = 'gzip')